# Agent RAG pour la Jurisprudence Constitutionnelle Française

Agent DSPy spécialisé dans l'analyse de la jurisprudence du Conseil constitutionnel.

## Architecture

| Composant | Rôle |
|-----------|------|
| **Retriever ColBERT** | Recherche sémantique dense (`maastrichtlawtech/colbert-legal-french`) |
| **LLM Générateur** | Synthèse des réponses (Mistral, Qwen, etc.) |

## Outils de l'agent

| Outil | Description |
|-------|-------------|
| `search_legal_docs` | Recherche N chunks (ordre aléatoire pour reranking) |
| `lookup_chunk` | Récupère un chunk avec contexte environnant |
| `lookup_legal_doc` | Récupère le document complet |

# 1. Préparation de l'environnement

Installation automatique des dépendances si `legal_rag` n'est pas installé.

In [1]:

# Install dependencies only if legal_rag is not already installed.
import os
import shutil
import subprocess
import sys
from importlib.util import find_spec
from pathlib import Path

# Enable CPU fallback for unsupported MPS ops (must be set before PyTorch import).
os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "1")

if find_spec("legal_rag") is None:
    REQ_URL = "https://github.com/artefactory-argimi/legal_rag/releases/download/data-juri-v1/requirements.txt"
    REPO_URL = "https://github.com/artefactory-argimi/legal_rag.git"
    UV_BIN = shutil.which("uv")
    PIP_CMD = [UV_BIN, "pip"] if UV_BIN else [sys.executable, "-m", "pip"]
    subprocess.check_call([*PIP_CMD, "install", "-r", REQ_URL])
    subprocess.check_call([*PIP_CMD, "install", "--no-deps", f"git+{REPO_URL}"])

import legal_rag as _  # noqa: F401

# 2. Configuration de l'Agent

Modifiez les paramètres ci-dessous pour adapter l'agent à votre environnement.

| Paramètre | Description |
|-----------|-------------|
| `GENERATOR_API_BASE` | Endpoint LLM (local `localhost:8000` ou Hugging Face) |
| `GENERATOR_MODEL_ID` | Modèle de génération (Mistral, Qwen, etc.) |
| `SEARCH_K` | Nombre de chunks récupérés pour le reranking |
| `TEMPERATURE` | Créativité (0 = déterministe, 1 = créatif) |
| `MAX_ITERS` | Iterations max de l'agent (recherche + lookup) |

In [2]:
import os

# === Paramètres du Générateur (LLM) ===
DEFAULT_TOKEN = os.environ.get("HF_API_TOKEN", "")
GENERATOR_API_KEY = DEFAULT_TOKEN  # @param {type:"string"}
GENERATOR_API_BASE = "http://localhost:8000/v1"  # @param ["https://router.huggingface.co/v1", "https://api.openai.com/v1", "http://localhost:8000/v1"]  # noqa: E501
GENERATOR_MODEL_ID = "mistralai/Magistral-Small-2509"  # @param ["mistralai/Magistral-Small-2509", "Qwen/Qwen3-4B-Thinking-2507", "HuggingFaceTB/SmolLM3-3B"]  # noqa: E501

# === Paramètres du Retriever (ColBERT) ===
ENCODER_MODEL_ID = "maastrichtlawtech/colbert-legal-french"  # @param {type:"string"}
INDEX_URI = "https://github.com/artefactory-argimi/legal_rag/releases/download/data-juri-v1/index.zip"  # @param {type:"string"}
INDEX_PATH = "./downloads/index_legal_constit/"  # @param {type:"string"}
INDEX_NAME = "colbert_legal_french_constit_index"  # @param {type:"string"}
DATASET_CONFIG = "constit"  # @param ["constit", "juri", "cetat"] {type:"string"}
SEARCH_K = 20  # @param {type:"integer"}

# === Paramètres de Génération ===
TEMPERATURE = 0.2  # @param {type:"number"}
MAX_ITERS = 4  # @param {type:"integer"}

configured_index = None

# 3. Validation et Préparation

Cette section télécharge l'index ColBERT et vérifie la connectivité du serveur LLM.

In [3]:
from pathlib import Path

import httpx
from legal_rag.assets import extract_zip, fetch_zip
from rich import print as rprint


def download_index(uri: str, path: Path, name: str) -> None:
    """Download and extract the index if not present."""
    root = path.expanduser() / name
    if root.exists():
        return
    path.mkdir(parents=True, exist_ok=True)
    zip_path = path.parent / "_idx.zip"
    try:
        extract_zip(fetch_zip(uri, zip_path), path)
    finally:
        zip_path.unlink(missing_ok=True)


def validate_index(path: Path, name: str) -> Path:
    """Validate PLAID index structure."""
    root = path / name
    fast_plaid = root / "fast_plaid_index"
    if not root.exists():
        raise FileNotFoundError(f"Index missing: {root}")
    if not fast_plaid.exists() or not (fast_plaid / "metadata.json").exists():
        raise FileNotFoundError(f"Invalid index: {root}")
    if not list(fast_plaid.glob("*.codes.npy")):
        raise FileNotFoundError(f"Incomplete index: {fast_plaid}")
    return root


def validate_llm() -> None:
    """Validate LLM server connectivity."""
    if "localhost" in GENERATOR_API_BASE or "127.0.0.1" in GENERATOR_API_BASE:
        base = GENERATOR_API_BASE.rstrip("/").rsplit("/v1", 1)[0]
        r = httpx.get(f"{base}/health", timeout=5)
        if r.status_code != 200:
            raise ConnectionError(f"LLM unhealthy: {r.status_code}")
    elif "huggingface" in GENERATOR_API_BASE and not GENERATOR_API_KEY:
        raise ValueError("HF_API_TOKEN required")


download_index(INDEX_URI, Path(INDEX_PATH), INDEX_NAME)
configured_index = validate_index(Path(INDEX_PATH), INDEX_NAME)
validate_llm()
rprint(f"[green]✓[/] Index: {configured_index.name}, LLM: {GENERATOR_MODEL_ID}")

✓ Index: colbert_legal_french_constit_index, LLM: mistralai/Magistral-Small-2509

In [4]:
import os
from typing import Iterable

from etils import ecolab

ecolab.auto_display()
ecolab.auto_inspect()

# Avoid optional vision deps when loading text models.
os.environ.setdefault("TRANSFORMERS_NO_TORCHVISION", "1")
os.environ.setdefault("DISABLE_TRANSFORMERS_AV", "1")
os.environ.setdefault("HF_HUB_TIMEOUT", "60")

/Users/hicham.randrianarivo/Work/demo_legal_rag/.venv/lib/python3.11/site-packages/google/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


'60'

# 4. Construction de l'Agent RAG

L'agent utilise le workflow défini dans `LegalRAGSignature` :

```
Recherche → Reranking → Validation → Formulation (→ Reformulation si nécessaire)
```

Les instructions et le comportement sont encodés dans les signatures DSPy, pas dans des prompts manuels.

In [5]:
from legal_rag.agent import build_agent

generator_api_key = GENERATOR_API_KEY or None
generator_api_base = (GENERATOR_API_BASE or "https://router.huggingface.co/v1").rstrip(
    "/"
)
clean_model_id = GENERATOR_MODEL_ID

agent = build_agent(
    student_model=clean_model_id,
    encoder_model=ENCODER_MODEL_ID,
    generator_api_key=generator_api_key,
    generator_api_base=generator_api_base,
    index_folder=configured_index.parent,  # PLAID expects the parent folder
    index_name=configured_index.name,  # Index name is the folder containing fast_plaid_index
    search_k=SEARCH_K,
    temperature=TEMPERATURE,
    max_iters=MAX_ITERS,
    dataset_config=DATASET_CONFIG,
)

[legal_rag] Loading encoder from maastrichtlawtech/colbert-legal-french


No sentence-transformers model found with name maastrichtlawtech/colbert-legal-french.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Tokenizer vocab size: 32007
Embedding layer size: 32007
Query prefix '[unused0]': ID 32006
Document prefix '[unused1]': ID 32007

⚠️  Token IDs exceed embedding size!
Resizing from 32007 to 32008
✓ Resized to 32008
✓ All token IDs are now valid
[legal_rag] Loading index from downloads/index_legal_constit (name=colbert_legal_french_constit_index)
[legal_rag] FastPLAID constrained to a single CPU worker.
[legal_rag] PLAID index status is_indexed=True


Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

# 5. Démonstration : Questions et Réponses

Nous testons l'agent avec deux types de questions :

| Question | Type | Comportement attendu |
|----------|------|---------------------|
| Agriculture | Hors périmètre | Refus poli sans appel d'outils |
| QPC Eric RAOULT | Dans le périmètre | Recherche → Reranking → Lookup → Réponse structurée |

## Définition des questions de test

Modifiez cette liste pour tester d'autres questions sur la jurisprudence constitutionnelle.

In [6]:
queries: Iterable[str] | str = [
    # Question hors périmètre - l'agent doit refuser poliment
    "Comment régler l'arrosage goutte-à-goutte de tomates en serre pendant une canicule ?",
    # Question dans le périmètre - l'agent doit rechercher et répondre
    "Quelle requête M. Eric RAOULT a-t-il présentée au Conseil constitutionnel",
]

## Exécution et résultats

Pour chaque question, l'agent retourne :
- `prediction.answer` : La réponse textuelle générée
- `prediction.trajectory` : L'historique des appels d'outils (recherche, reranking, lookup)

In [7]:
for idx, question in enumerate(queries, start=1):
    rprint(f"[bold cyan]Question {idx}[/]: {question}")

    prediction = agent(question=question)
    rprint(f"[bold green]Réponse[/]\n{prediction.answer}")
    rprint("[bold magenta]Trajectoire[/]")
    ecolab.json(prediction.trajectory)

Question 1: Comment régler l'arrosage goutte-à-goutte de tomates en serre pendant une canicule ?

Réponse
Le domaine demandé n'est pas couvert.

Trajectoire

Question 2: Quelle requête M. Eric RAOULT a-t-il présentée au Conseil constitutionnel

Réponse
**Requête présentée par M. Eric RAOULT au Conseil constitutionnel**

**Faits** : M. Eric RAOULT a présenté une requête enregistrée au secrétariat général du Conseil constitutionnel le 
12 juin 1997, tendant à l'annulation des opérations électorales pour la désignation d'un député à l'Assemblée 
nationale dans la 12ème circonscription de la Seine-Saint-Denis, qui se sont déroulées les 25 mai et 1er juin 1997.

**Fondement** : La requête était basée sur plusieurs griefs, notamment la violation de la loi du 19 juillet 1977 
modifiée relative à la publication et à la diffusion de certains sondages d'opinion, la diffusion irrégulière d'un 
tract, et des manoeuvres suggérant son absence au second tour.

**Dispositif** : Le Conseil constitutionnel a rejeté la requête de M. Eric RAOULT par décision du 15 janvier 1998 
(Décision n° 97-2330 AN).

**Articles cités** :
- Article 59 de la Constitution
- Ordonnance n° 58-1067 du 7 novembre 1958 modifiée portant loi organique sur le Conseil constitutionnel
- Loi du 29 juillet 1881 modifiée sur la liberté de la presse
- Code électoral
- Règlement applicable à la procédure suivie devant le Conseil constitutionnel pour le contentieux de l'élection 
des députés et des sénateurs

**Référence** : Décision du Conseil constitutionnel n° 97-2330 AN du 15 janvier 1998.

Trajectoire